In [1]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 133kB 5.3MB/s 
     |████████████████████████████████| 1.2MB 19.8MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 163kB 28.9MB/s 
     |████████████████████████████████| 184kB 23.1MB/s 
     |████████████████████████████████| 245kB 27.7MB/s 
     |████████████████████████████████| 296kB 24.4MB/s 
     |████████████████████████████████| 829kB 27.3MB/s 
     |████████████████████████████████| 122kB 39.9MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 
     |████████████████████████████████| 460kB 37.6MB/s 
     |████████████████████████████████| 31.9MB 260kB/s 
     |████████████████████████████████| 92kB 3.2MB/s 
     |████████████████████████████████| 153kB 54.4MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 1.3MB 49.4MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 102kB 11.1MB/s 

In [1]:
!pip3 install --upgrade pip setuptools wheel

     |████████████████████████████████| 1.6MB 15.5MB/s 
     |████████████████████████████████| 829kB 46.2MB/s 
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Found existing installation: setuptools 56.1.0
    Uninstalling setuptools-56.1.0:
      Successfully uninstalled setuptools-56.1.0


In [3]:
import requests
import os
if 'TPU_DRIVER_MODE' not in globals():
  url = 'http://' + os.environ['COLAB_TPU_ADDR'].split(':')[0] + ':8475/requestversion/tpu_driver0.1-dev20191206'
  resp = requests.post(url)
  TPU_DRIVER_MODE = 1

# The following is required to use TPU Driver as JAX's backend.
from jax.config import config
config.FLAGS.jax_xla_backend = "tpu_driver"
config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']
print(config.FLAGS.jax_backend_target)

grpc://10.83.236.114:8470


In [2]:
from jax.lib import xla_bridge
xla_bridge.get_backend().devices()
xla_bridge.xla_client.get_local_backend().platform

'gpu'

In [ ]:
!pip3 install --upgrade jax jaxlib==0.1.65+cuda111 -f https://storage.googleapis.com/jax-releases/jax_releases.html

Looking in links: https://storage.googleapis.com/jax-releases/jax_releases.html
     |████████████████████████████████| 189.4 MB 11 kB/s 
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.1.57+cuda111
    Uninstalling jaxlib-0.1.57+cuda111:
      Successfully uninstalled jaxlib-0.1.57+cuda111


In [3]:
!unzip gated_linear_networks.zip

Archive:  gated_linear_networks.zip
   creating: gated_linear_networks/
  inflating: __MACOSX/._gated_linear_networks  
  inflating: gated_linear_networks/.DS_Store  
  inflating: __MACOSX/gated_linear_networks/._.DS_Store  
  inflating: gated_linear_networks/requirements.txt  
  inflating: __MACOSX/gated_linear_networks/._requirements.txt  
  inflating: gated_linear_networks/my_example.py  
   creating: gated_linear_networks/colabs/
  inflating: __MACOSX/gated_linear_networks/._colabs  
   creating: gated_linear_networks/__pycache__/
  inflating: __MACOSX/gated_linear_networks/.___pycache__  
  inflating: gated_linear_networks/run.sh  
  inflating: __MACOSX/gated_linear_networks/._run.sh  
  inflating: gated_linear_networks/README.md  
  inflating: __MACOSX/gated_linear_networks/._README.md  
   creating: gated_linear_networks/examples/
  inflating: __MACOSX/gated_linear_networks/._examples  
  inflating: gated_linear_networks/bernoulli_test.py  
  inflating: __MACOSX/gated_linear_net

In [7]:
!pip3 install --upgrade tf_slim

     |████████████████████████████████| 352 kB 9.1 MB/s 


In [4]:
!pip3 install chex
!pip3 install dm-haiku
!pip3 install rlax

     |████████████████████████████████| 52 kB 877 kB/s 
     |████████████████████████████████| 284 kB 13.3 MB/s 
     |████████████████████████████████| 110 kB 14.1 MB/s 


In [6]:
"""Haiku modules for feature processing."""

import copy
from typing import Tuple

import chex
import haiku as hk
import jax.numpy as jnp
import numpy as np
from scipy.ndimage import interpolation
import tensorflow_datasets as tfds

Array = chex.Array


def _moments(image):
  """Compute the first and second moments of a given image."""
  c0, c1 = np.mgrid[:image.shape[0], :image.shape[1]]
  total_image = np.sum(image)
  m0 = np.sum(c0 * image) / total_image
  m1 = np.sum(c1 * image) / total_image
  m00 = np.sum((c0 - m0)**2 * image) / total_image
  m11 = np.sum((c1 - m1)**2 * image) / total_image
  m01 = np.sum((c0 - m0) * (c1 - m1) * image) / total_image
  mu_vector = np.array([m0, m1])
  covariance_matrix = np.array([[m00, m01], [m01, m11]])
  return mu_vector, covariance_matrix


def _deskew(image):
  """Image deskew."""
  c, v = _moments(image)
  alpha = v[0, 1] / v[0, 0]
  affine = np.array([[1, 0], [alpha, 1]])
  ocenter = np.array(image.shape) / 2.0
  offset = c - np.dot(affine, ocenter)
  return interpolation.affine_transform(image, affine, offset=offset)


def _deskew_dataset(dataset):
  """Dataset deskew."""
  deskewed = copy.deepcopy(dataset)
  for k, before in dataset.items():
    images = before["image"]
    num_images = images.shape[0]
    after = np.stack([_deskew(i) for i in np.squeeze(images, axis=-1)], axis=0)
    deskewed[k]["image"] = np.reshape(after, (num_images, -1))
  return deskewed


def load_deskewed_mnist(*a, **k):
  """Returns deskewed MNIST numpy dataset."""
  mnist_data, info = tfds.load(*a, **k)
  mnist_data = tfds.as_numpy(mnist_data)
  deskewed_data = _deskew_dataset(mnist_data)
  return deskewed_data, info


class MeanStdEstimator(hk.Module):
  """Online mean and standard deviation estimator using Welford's algorithm."""

  def __call__(self, sample: jnp.DeviceArray) -> Tuple[Array, Array]:
    if len(sample.shape) > 1:
      raise ValueError("sample must be a rank 0 or 1 DeviceArray.")

    count = hk.get_state("count", shape=(), dtype=jnp.int32, init=jnp.zeros)
    mean = hk.get_state(
        "mean", shape=sample.shape, dtype=jnp.float32, init=jnp.zeros)
    m2 = hk.get_state(
        "m2", shape=sample.shape, dtype=jnp.float32, init=jnp.zeros)

    count += 1
    delta = sample - mean
    mean += delta / count
    delta_2 = sample - mean
    m2 += delta * delta_2

    hk.set_state("count", count)
    hk.set_state("mean", mean)
    hk.set_state("m2", m2)

    stddev = jnp.sqrt(m2 / count)
    return mean, stddev

In [2]:
"""Bernoulli Gated Linear Network."""

from typing import List, Text, Tuple

import chex
import jax
import jax.numpy as jnp
import rlax
import tensorflow_probability as tfp

from gated_linear_networks import base

#tfp = tfp.experimental.substrates.jax
#tfd = tfp.distributions

Array = chex.Array

GLN_EPS = 0.01
MAX_WEIGHT = 200.


class GatedLinearNetwork(base.GatedLinearNetwork):
  """Bernoulli Gated Linear Network."""

  def __init__(self,
               output_sizes: List[int],
               context_dim: int,
               name: Text = "bernoulli_gln"):
    """Initialize a Bernoulli GLN."""
    super(GatedLinearNetwork, self).__init__(
        output_sizes,
        context_dim,
        inference_fn=GatedLinearNetwork._inference_fn,
        update_fn=GatedLinearNetwork._update_fn,
        init=jnp.zeros,
        dtype=jnp.float32,
        name=name)

  def _add_bias(self, inputs):
    return jnp.append(inputs, rlax.sigmoid(1.))

  @staticmethod
  def _inference_fn(
      inputs: Array,           # [input_size]
      side_info: Array,        # [side_info_size]
      weights: Array,          # [2**context_dim, input_size]
      hyperplanes: Array,      # [context_dim, side_info_size]
      hyperplane_bias: Array,  # [context_dim]
  ) -> Array:
    """Inference step for a single Beurnolli neuron."""

    weight_index = GatedLinearNetwork._compute_context(side_info, hyperplanes,
                                                       hyperplane_bias)
    used_weights = weights[weight_index]
    inputs = rlax.logit(jnp.clip(inputs, GLN_EPS, 1. - GLN_EPS))
    prediction = rlax.sigmoid(jnp.dot(used_weights, inputs))

    return prediction

  @staticmethod
  def _update_fn(
      inputs: Array,           # [input_size]
      side_info: Array,        # [side_info_size]
      weights: Array,          # [2**context_dim, num_features]
      hyperplanes: Array,      # [context_dim, side_info_size]
      hyperplane_bias: Array,  # [context_dim]
      target: Array,           # []
      learning_rate: float,
  ) -> Tuple[Array, Array, Array]:
    """Update step for a single Bernoulli neuron."""

    def log_loss_fn(inputs, side_info, weights, hyperplanes, hyperplane_bias,
                    target):
      """Log loss for a single Bernoulli neuron."""
      prediction = GatedLinearNetwork._inference_fn(inputs, side_info, weights,
                                                    hyperplanes,
                                                    hyperplane_bias)
      prediction = jnp.clip(prediction, GLN_EPS, 1. - GLN_EPS)
      return rlax.log_loss(prediction, target), prediction

    grad_log_loss = jax.value_and_grad(log_loss_fn, argnums=2, has_aux=True)
    ((log_loss, prediction),
     dloss_dweights) = grad_log_loss(inputs, side_info, weights, hyperplanes,
                                     hyperplane_bias, target)

    delta_weights = learning_rate * dloss_dweights
    new_weights = jnp.clip(weights - delta_weights, -MAX_WEIGHT, MAX_WEIGHT)
    return new_weights, prediction, log_loss


class LastNeuronAggregator(base.LastNeuronAggregator):
  """Bernoulli last neuron aggregator, implemented by the super class."""
  pass


In [58]:
#from absl import app
#from absl import flags

import haiku as hk
import jax
import jax.numpy as jnp
import rlax

#from gated_linear_networks import bernoulli
#from gated_linear_networks.examples import utils
#import bernoulli
#from examples import utils

MAX_TRAIN_STEPS = 50

# Small example network, achieves ~95% test set accuracy =======================
# Network parameters.
NUM_LAYERS = 2

NEURONS_PER_LAYER = 70

CONTEXT_DIM = 1


# Learning rate schedule.
MAX_LR = 0.003

LR_CONSTANT = 1.0

LR_DECAY = 0.1


# Logging parameters.
#EVALUATE_EVERY = 1000
EVALUATE_EVERY = 10


def main(unused_argv):
  # Load MNIST dataset =========================================================
  mnist_data, info = load_deskewed_mnist(
      name='mnist', batch_size=-1, with_info=True)
  num_classes = info.features['label'].num_classes

  (train_images, train_labels) = (mnist_data['train']['image'],
                                  mnist_data['train']['label'])

  (test_images, test_labels) = (mnist_data['test']['image'],
                                mnist_data['test']['label'])
  print(train_images.shape)
  print(train_labels.shape)

  # Build a (binary) GLN classifier ============================================
  def network_factory():

    def gln_factory():
      output_sizes = [NEURONS_PER_LAYER] * NUM_LAYERS + [1]
      return GatedLinearNetwork(
          output_sizes=output_sizes, context_dim=CONTEXT_DIM)

    return LastNeuronAggregator(gln_factory)

  def extract_features(image):
    mean, stddev = MeanStdEstimator()(image)
    standardized_img = (image - mean) / (stddev + 1.)
    inputs = rlax.sigmoid(standardized_img)
    side_info = standardized_img
    return inputs, side_info

  def inference_fn(image, *args, **kwargs):
    inputs, side_info = extract_features(image)
    return network_factory().inference(inputs, side_info, *args, **kwargs)

  def update_fn(image, *args, **kwargs):
    inputs, side_info = extract_features(image)
    return network_factory().update(inputs, side_info, *args, **kwargs)

  init_, inference_ = hk.without_apply_rng(
      hk.transform_with_state(inference_fn))
  _, update_ = hk.without_apply_rng(hk.transform_with_state(update_fn))

  # Map along class dimension to create a one-vs-all classifier ================
  @jax.jit
  def init(dummy_image, key):
    """One-vs-all classifier init fn."""
    dummy_images = jnp.stack([dummy_image] * num_classes, axis=0)
    keys = jax.random.split(key, num_classes)
    return jax.vmap(init_, in_axes=(0, 0))(keys, dummy_images)

  @jax.jit
  def accuracy(params, state, image, label):
    """One-vs-all classifier inference fn."""
    fn = jax.vmap(inference_, in_axes=(0, 0, None))
    predictions, unused_state = fn(params, state, image)
    return (jnp.argmax(predictions) == label).astype(jnp.float32)

  @jax.jit
  def update(params, state, step, image, label):
    """One-vs-all classifier update fn."""

    # Learning rate schedules.
    learning_rate = jnp.minimum(
        MAX_LR, LR_CONSTANT / (1. + LR_DECAY * step))

    # Update weights and report log-loss.
    targets = hk.one_hot(jnp.asarray(label), num_classes)

    fn = jax.vmap(update_, in_axes=(0, 0, None, 0, None))
    out = fn(params, state, image, targets, learning_rate)
    (params, unused_predictions, log_loss), state = out
    return (jnp.mean(log_loss), params), state

  # Train on train split =======================================================
  dummy_image = train_images[0]
  params, state = init(dummy_image, jax.random.PRNGKey(42))

  for step, (image, label) in enumerate(zip(train_images, train_labels), 1):
    (unused_loss, params), state = update(
        params,
        state,
        step,
        image,
        label,
    )

    # Evaluate on test split ===================================================
    if not step % EVALUATE_EVERY:
      batch_accuracy = jax.vmap(accuracy, in_axes=(None, None, 0, 0))
      accuracies = batch_accuracy(params, state, test_images, test_labels)
      total_accuracy = float(jnp.mean(accuracies))

      # Report statistics.
      print({
          'step': step,
          'accuracy': float(total_accuracy),
      })

    if MAX_TRAIN_STEPS is not None and step >= MAX_TRAIN_STEPS:
      return



main(0)

(60000, 784)
(60000,)
{'step': 10, 'accuracy': 0.15040001273155212}
{'step': 20, 'accuracy': 0.2200000137090683}
{'step': 30, 'accuracy': 0.21980001032352448}
{'step': 40, 'accuracy': 0.5315999984741211}
{'step': 50, 'accuracy': 0.5707000494003296}


In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
from absl import app
from absl import flags
import numpy as np
import os
import tensorflow as tf
from hparams import HParams
from matplotlib import pyplot as plt

from bandits.algorithms.bootstrapped_bnn_sampling import BootstrappedBNNSampling
from bandits.core.contextual_bandit import run_contextual_bandit
from bandits.data.data_sampler import sample_adult_data
from bandits.data.data_sampler import sample_census_data
from bandits.data.data_sampler import sample_covertype_data
from bandits.data.data_sampler import sample_jester_data
from bandits.data.data_sampler import sample_mushroom_data
from bandits.data.data_sampler import sample_statlog_data
from bandits.data.data_sampler import sample_stock_data
from bandits.algorithms.fixed_policy_sampling import FixedPolicySampling
from bandits.algorithms.linear_full_posterior_sampling import LinearFullPosteriorSampling
from bandits.algorithms.neural_linear_sampling import NeuralLinearPosteriorSampling
from bandits.algorithms.parameter_noise_sampling import ParameterNoiseSampling
from bandits.algorithms.posterior_bnn_sampling import PosteriorBNNSampling
from bandits.data.synthetic_data_sampler import sample_linear_data
from bandits.data.synthetic_data_sampler import sample_sparse_linear_data
from bandits.data.synthetic_data_sampler import sample_wheel_bandit_data
from bandits.algorithms.uniform_sampling import UniformSampling

import haiku as hk
import jax
import jax.numpy as jnp
import rlax

#from gated_linear_networks import bernoulli
#from gated_linear_networks.examples import utils
#import bernoulli
#from examples import utils

MAX_TRAIN_STEPS = 2000

# Small example network, achieves ~95% test set accuracy =======================
# Network parameters.
NUM_LAYERS = 2

NEURONS_PER_LAYER = 70

CONTEXT_DIM = 1


# Learning rate schedule.
MAX_LR = 0.003

LR_CONSTANT = 1.0

LR_DECAY = 0.1


# Logging parameters.
#EVALUATE_EVERY = 1000
EVALUATE_EVERY = 1

# Set up your file routes to the data files.
base_route = os.getcwd()
data_route = 'contextual_bandits/datasets'

FLAGS = flags.FLAGS
FLAGS.set_default('alsologtostderr', True)
flags.DEFINE_string('logdir', '/tmp/bandits/', 'Base directory to save output')
flags.DEFINE_string(
    'mushroom_data',
    os.path.join(base_route, data_route, 'mushroom.data'),
    'Directory where Mushroom data is stored.')
flags.DEFINE_string(
    'financial_data',
    os.path.join(base_route, data_route, 'raw_stock_contexts'),
    'Directory where Financial data is stored.')
flags.DEFINE_string(
    'jester_data',
    os.path.join(base_route, data_route, 'jester_data_40jokes_19181users.npy'),
    'Directory where Jester data is stored.')
flags.DEFINE_string(
    'statlog_data',
    os.path.join(base_route, data_route, 'shuttle.trn'),
    'Directory where Statlog data is stored.')
flags.DEFINE_string(
    'adult_data',
    os.path.join(base_route, data_route, 'adult.full'),
    'Directory where Adult data is stored.')
flags.DEFINE_string(
    'covertype_data',
    os.path.join(base_route, data_route, 'covtype.data'),
    'Directory where Covertype data is stored.')
flags.DEFINE_string(
    'census_data',
    os.path.join(base_route, data_route, 'USCensus1990.data.txt'),
    'Directory where Census data is stored.')


def sample_data(data_type, num_contexts=None):
  """Sample data from given 'data_type'.

  Args:
    data_type: Dataset from which to sample.
    num_contexts: Number of contexts to sample.

  Returns:
    dataset: Sampled matrix with rows: (context, reward_1, ..., reward_num_act).
    opt_rewards: Vector of expected optimal reward for each context.
    opt_actions: Vector of optimal action for each context.
    num_actions: Number of available actions.
    context_dim: Dimension of each context.
  """

  if data_type == 'linear':
    # Create linear dataset
    num_actions = 8
    context_dim = 10
    noise_stds = [0.01 * (i + 1) for i in range(num_actions)]
    dataset, _, opt_linear = sample_linear_data(num_contexts, context_dim,
                                                num_actions, sigma=noise_stds)
    opt_rewards, opt_actions = opt_linear
  elif data_type == 'sparse_linear':
    # Create sparse linear dataset
    num_actions = 7
    context_dim = 10
    noise_stds = [0.01 * (i + 1) for i in range(num_actions)]
    num_nnz_dims = int(context_dim / 3.0)
    dataset, _, opt_sparse_linear = sample_sparse_linear_data(
        num_contexts, context_dim, num_actions, num_nnz_dims, sigma=noise_stds)
    opt_rewards, opt_actions = opt_sparse_linear
  elif data_type == 'mushroom':
    # Create mushroom dataset
    num_actions = 2
    context_dim = 117
    file_name = FLAGS.mushroom_data
    dataset, opt_mushroom = sample_mushroom_data(file_name, num_contexts)
    opt_rewards, opt_actions = opt_mushroom
  elif data_type == 'financial':
    num_actions = 8
    context_dim = 21
    num_contexts = min(3713, num_contexts)
    noise_stds = [0.01 * (i + 1) for i in range(num_actions)]
    file_name = FLAGS.financial_data
    dataset, opt_financial = sample_stock_data(file_name, context_dim,
                                               num_actions, num_contexts,
                                               noise_stds, shuffle_rows=True)
    opt_rewards, opt_actions = opt_financial
  elif data_type == 'jester':
    num_actions = 8
    context_dim = 32
    num_contexts = min(19181, num_contexts)
    file_name = FLAGS.jester_data
    dataset, opt_jester = sample_jester_data(file_name, context_dim,
                                             num_actions, num_contexts,
                                             shuffle_rows=True,
                                             shuffle_cols=True)
    opt_rewards, opt_actions = opt_jester
  elif data_type == 'statlog':
    file_name = FLAGS.statlog_data
    num_actions = 7
    num_contexts = min(43500, num_contexts)
    sampled_vals = sample_statlog_data(file_name, num_contexts,
                                       shuffle_rows=True)
    contexts, rewards, (opt_rewards, opt_actions) = sampled_vals
    dataset = np.hstack((contexts, rewards))
    context_dim = contexts.shape[1]
  elif data_type == 'adult':
    file_name = FLAGS.adult_data
    num_actions = 14
    num_contexts = min(45222, num_contexts)
    sampled_vals = sample_adult_data(file_name, num_contexts,
                                     shuffle_rows=True)
    contexts, rewards, (opt_rewards, opt_actions) = sampled_vals
    dataset = np.hstack((contexts, rewards))
    context_dim = contexts.shape[1]
  elif data_type == 'covertype':
    file_name = FLAGS.covertype_data
    num_actions = 7
    num_contexts = min(150000, num_contexts)
    sampled_vals = sample_covertype_data(file_name, num_contexts,
                                         shuffle_rows=True)
    contexts, rewards, (opt_rewards, opt_actions) = sampled_vals
    dataset = np.hstack((contexts, rewards))
    context_dim = contexts.shape[1]
  elif data_type == 'census':
    file_name = FLAGS.census_data
    num_actions = 9
    num_contexts = min(150000, num_contexts)
    sampled_vals = sample_census_data(file_name, num_contexts,
                                      shuffle_rows=True)
    contexts, rewards, (opt_rewards, opt_actions) = sampled_vals
    dataset = np.hstack((contexts, rewards))
    context_dim = contexts.shape[1]
  elif data_type == 'wheel':
    delta = 0.95
    num_actions = 5
    context_dim = 2
    mean_v = [1.0, 1.0, 1.0, 1.0, 1.2]
    std_v = [0.05, 0.05, 0.05, 0.05, 0.05]
    mu_large = 50
    std_large = 0.01
    dataset, opt_wheel = sample_wheel_bandit_data(num_contexts, delta,
                                                  mean_v, std_v,
                                                  mu_large, std_large)
    opt_rewards, opt_actions = opt_wheel

  return dataset, opt_rewards, opt_actions, num_actions, context_dim


def display_results(algos, opt_rewards, opt_actions, h_rewards, t_init, name):
  """Displays summary statistics of the performance of each algorithm."""

  print('---------------------------------------------------')
  print('---------------------------------------------------')
  print('{} bandit completed after {} seconds.'.format(
    name, time.time() - t_init))
  print('---------------------------------------------------')

  performance_pairs = []
  for j, a in enumerate(algos):
    performance_pairs.append((a.name, np.sum(h_rewards[:, j])))
  performance_pairs = sorted(performance_pairs,
                             key=lambda elt: elt[1],
                             reverse=True)
  for i, (name, reward) in enumerate(performance_pairs):
    print('{:3}) {:20}| \t \t total reward = {:10}.'.format(i, name, reward))

  print('---------------------------------------------------')
  print('Optimal total reward = {}.'.format(np.sum(opt_rewards)))
  print('Frequency of optimal actions (action, frequency):')
  print([[elt, list(opt_actions).count(elt)] for elt in set(opt_actions)])
  print('---------------------------------------------------')
  print('---------------------------------------------------')


def main(_):

  # Problem parameters
  num_contexts = 2000
  num_classes = 14
  # Data type in {linear, sparse_linear, mushroom, financial, jester,
  #                 statlog, adult, covertype, census, wheel}
  data_type = 'adult'

  # Create dataset
  sampled_vals = sample_data(data_type, num_contexts)
  dataset, opt_rewards, opt_actions, num_actions, context_dim = sampled_vals
  # dataset: (num_contexts, context_dim + num_actions)
  # opt_rewards: (num_contexts,)
  dataset = dataset[:, :num_contexts]
  labels = opt_actions

  # Build a (binary) GLN classifier ============================================
  def network_factory():

    def gln_factory():
      output_sizes = [NEURONS_PER_LAYER] * NUM_LAYERS + [1]
      return GatedLinearNetwork(
          output_sizes=output_sizes, context_dim=CONTEXT_DIM)

    return LastNeuronAggregator(gln_factory)

  def extract_features(image):
    mean, stddev = MeanStdEstimator()(image)
    standardized_img = (image - mean) / (stddev + 1.)
    inputs = rlax.sigmoid(standardized_img)
    side_info = standardized_img
    return inputs, side_info

  def inference_fn(image, *args, **kwargs):
    inputs, side_info = extract_features(image)
    return network_factory().inference(inputs, side_info, *args, **kwargs)

  def update_fn(image, *args, **kwargs):
    inputs, side_info = extract_features(image)
    return network_factory().update(inputs, side_info, *args, **kwargs)

  init_, inference_ = hk.without_apply_rng(
      hk.transform_with_state(inference_fn))
  _, update_ = hk.without_apply_rng(hk.transform_with_state(update_fn))

  # Map along class dimension to create a one-vs-all classifier ================
  @jax.jit
  def init(dummy_image, key):
    """One-vs-all classifier init fn."""
    dummy_images = jnp.stack([dummy_image] * num_classes, axis=0)
    keys = jax.random.split(key, num_classes)
    return jax.vmap(init_, in_axes=(0, 0))(keys, dummy_images)

  @jax.jit
  def accuracy(params, state, image, label):
    """One-vs-all classifier inference fn."""
    fn = jax.vmap(inference_, in_axes=(0, 0, None))
    predictions, unused_state = fn(params, state, image)
    return (jnp.argmax(predictions) == label).astype(jnp.float32)

  @jax.jit
  def update(params, state, step, image, label):
    """One-vs-all classifier update fn."""

    # Learning rate schedules.
    learning_rate = jnp.minimum(
        MAX_LR, LR_CONSTANT / (1. + LR_DECAY * step))

    # Update weights and report log-loss.
    targets = hk.one_hot(jnp.asarray(label), num_classes)

    fn = jax.vmap(update_, in_axes=(0, 0, None, 0, None))
    out = fn(params, state, image, targets, learning_rate)
    (params, unused_predictions, log_loss), state = out
    return (jnp.mean(log_loss), params), state

  # Train on train split =======================================================
  #dummy_image = train_images[0]
  dummy_image = dataset[0]
  params, state = init(dummy_image, jax.random.PRNGKey(42))
  total_rewards = 0.

  for step, (image, label) in enumerate(zip(dataset, labels), 1):
    (unused_loss, params), state = update(
        params,
        state,
        step,
        image,
        label,
    )

    # Evaluate on test split ===================================================
    if not step % EVALUATE_EVERY:
      batch_accuracy = jax.vmap(accuracy, in_axes=(None, None, 0, 0))
      #accuracies = batch_accuracy(params, state, test_images, test_labels)
      reward = float(jnp.mean(batch_accuracy(params, state, image, label)))
      total_rewards += reward
      #total_accuracy = float(jnp.mean(accuracies))

      # Report statistics.

    if MAX_TRAIN_STEPS is not None and step >= MAX_TRAIN_STEPS:
      return



if __name__ == '__main__':
  app.run(main)

Instructions for updating:
non-resource variables are not supported in the long term


FATAL Flags parsing error: Unknown command line flag 'f'
Pass --helpshort or --helpfull to see help on flags.


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
!python contextual_bandit_GLN.py

Instructions for updating:
non-resource variables are not supported in the long term
tcmalloc: large alloc 1356980224 bytes == 0x556a49aa8000 @  0x7f0a6599e1e7 0x7f0a52fd346e 0x7f0a53023c7b 0x7f0a5302435f 0x7f0a530c6103 0x55699fc1fd54 0x55699fc1fa50 0x55699fc94105 0x55699fc2130a 0x55699fc8f3b5 0x55699fc8e4ae 0x55699fc213ea 0x55699fc8f3b5 0x55699fc2130a 0x55699fc8f3b5 0x55699fc2130a 0x55699fc8f3b5 0x55699fc8e4ae 0x55699fc213ea 0x55699fc9032a 0x55699fc8e7ad 0x55699fc213ea 0x55699fc9032a 0x55699fc8e7ad 0x55699fc21c9f 0x55699fc62d79 0x55699fc5fcc4 0x55699fc20559 0x55699fc944f8 0x55699fc8e4ae 0x55699fc213ea
I0525 22:44:24.434667 139682635380608 utils.py:157] NumExpr defaulting to 4 threads.
tcmalloc: large alloc 1356980224 bytes == 0x556a9a8c6000 @  0x7f0a6599e1e7 0x7f0a52fd346e 0x7f0a53023c7b 0x7f0a5302435f 0x7f0a530c6103 0x55699fc1fd54 0x55699fc1fa50 0x55699fc94105 0x55699fc8e4ae 0x55699fc213ea 0x55699fc9032a 0x55699fc8e4ae 0x55699fc213ea 0x55699fc9032a 0x55699fc8e7ad 0x55699fc213ea 0x556

In [5]:
#!rm -r __MACOSX/
#!rm -r deep_contextual_bandits/
!unzip deep_contextual_bandits.zip

Archive:  deep_contextual_bandits.zip
   creating: deep_contextual_bandits/
  inflating: __MACOSX/._deep_contextual_bandits  
   creating: deep_contextual_bandits/contextual_bandits/
  inflating: __MACOSX/deep_contextual_bandits/._contextual_bandits  
  inflating: deep_contextual_bandits/.DS_Store  
  inflating: __MACOSX/deep_contextual_bandits/._.DS_Store  
  inflating: deep_contextual_bandits/hparams.py  
   creating: deep_contextual_bandits/bandits/
  inflating: __MACOSX/deep_contextual_bandits/._bandits  
   creating: deep_contextual_bandits/__pycache__/
  inflating: __MACOSX/deep_contextual_bandits/.___pycache__  
  inflating: deep_contextual_bandits/example_main.py  
  inflating: __MACOSX/deep_contextual_bandits/._example_main.py  
  inflating: deep_contextual_bandits/README.md  
  inflating: __MACOSX/deep_contextual_bandits/._README.md  
  inflating: deep_contextual_bandits/contextual_bandits/.DS_Store  
  inflating: __MACOSX/deep_contextual_bandits/contextual_bandits/._.DS_Stor

In [45]:
!python example_main.py

Instructions for updating:
non-resource variables are not supported in the long term
tcmalloc: large alloc 1356980224 bytes == 0x55e7dcac8000 @  0x7faca0f5f1e7 0x7fac9e24e46e 0x7fac9e29ec7b 0x7fac9e29f35f 0x7fac9e341103 0x55e733297d54 0x55e733297a50 0x55e73330c105 0x55e73329930a 0x55e7333073b5 0x55e7333064ae 0x55e7332993ea 0x55e7333073b5 0x55e73329930a 0x55e7333073b5 0x55e73329930a 0x55e7333073b5 0x55e7333064ae 0x55e7332993ea 0x55e73330832a 0x55e7333067ad 0x55e7332993ea 0x55e73330832a 0x55e7333067ad 0x55e733299c9f 0x55e7332dad79 0x55e7332d7cc4 0x55e733298559 0x55e73330c4f8 0x55e7333064ae 0x55e7332993ea
I0525 07:01:47.218484 140379415971712 utils.py:157] NumExpr defaulting to 4 threads.
tcmalloc: large alloc 1356980224 bytes == 0x55e82d8e6000 @  0x7faca0f5f1e7 0x7fac9e24e46e 0x7fac9e29ec7b 0x7fac9e29f35f 0x7fac9e341103 0x55e733297d54 0x55e733297a50 0x55e73330c105 0x55e7333064ae 0x55e7332993ea 0x55e73330832a 0x55e7333064ae 0x55e7332993ea 0x55e73330832a 0x55e7333067ad 0x55e7332993ea 0x55e